In [2]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from QuantumRingsLib import qasm2
from QuantumRingsLib import Parameter, ParameterVector
import json
from random import random, uniform
import numpy as np
import os

# Load the JSON data
def load_esg_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Setup the Hamiltonian based on ESG scores
def get_Paulioperator(esg_data):
    pauli_list = []
    weights = []

    # Create Pauli operators based on ESG scores
    for company in esg_data:
        score = company["esg_score"]
        weights.append(score)
        # Create a corresponding Pauli operator (assuming 4 companies)
        pauli_operator = "I" * 4  # Modify this to suit your encoding
        pauli_operator = pauli_operator[:len(pauli_operator)-len(weights)] + "Z" * len(weights)
        pauli_list.append([score, pauli_operator])

    return pauli_list

# Setting up Quantum Rings Provider
provider = QuantumRingsProvider(token=os.environ.get('TOKEN_QUANTUMRINGS'), name=os.environ.get('ACCOUNT_QUANTUMRINGS'))
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

print(provider.active_account())

# Define the ansatz (can use SimpleAnsatz or TwoLocalAnsatz)
def SimpleAnsatz(qc, q, n_qubits, theta_list, reps=5, insert_barriers=False):
    for i in range(reps + 1):
        for j in range(n_qubits):
            qc.ry(theta_list[(i * n_qubits) + j], q[j])
        if insert_barriers:
            qc.barrier()
    return

# Define perform_pauli_measurements function
def perform_pauli_measurements(qubitOp, param_dict, SHOTS=1024):
    avg = 0.0
    n_qubits = len(qubitOp[0][1])
    pauli_list = qubitOp

    for p in pauli_list:
        weight = p[0].real
        pauli = p[1]

        qc = QuantumCircuit(n_qubits)
        for i in range(n_qubits):
            qc.ry(param_dict[i], i)

        # Apply Pauli operators
        for i in range(n_qubits):
            if pauli[i] == "Y":
                qc.sdg(i)
                qc.h(i)
            elif pauli[i] == "X":
                qc.h(i)

        qc.measure_all()
        job = backend.run(qc, shots=SHOTS, mode="sync", performance="HighestEfficiency", quiet=True)
        job_monitor(job, quiet=True)

        results = job.result()
        result_dict = results.get_counts()

        # Measurement calculation
        measurement = sum((value if (bin(int(key, 2) & pauli_int).count("1") & 1) == 0 else -value) 
                           for key, value in result_dict.items())
        measurement /= SHOTS
        measurement *= weight
        avg += measurement

    return avg

# Load ESG data from JSON
esg_data = load_esg_data('./company_esgscore.json')
qubitOp = get_Paulioperator(esg_data)

# Initialize parameters
n_qubits = len(esg_data)
param_dict = np.random.rand((n_qubits + 1) * n_qubits)

# Run VQE optimization
for _ in range(100):  # You can adjust the number of iterations as needed
    avg_energy = perform_pauli_measurements(qubitOp, param_dict, SHOTS=shots)
    print(f"Current energy: {avg_energy}")

# Print the final optimized parameters
print("Optimized parameters:", param_dict)


{'name': 'challenge4_team2@qai-ventures.com', 'token': 'rings-256.BT5Mmd8rd5fPx5K6p0Ff9bVyA9cUDON3', 'max_qubits': '256'}


NameError: name 'pauli_int' is not defined

In [11]:
## this is for esg_score
## Code is missing to optimizet initial parameters input_params=output_params
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
import json
import numpy as np
import os

# Load the JSON data
def load_esg_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Setup the Hamiltonian based on ESG scores
def get_Paulioperator(esg_data):
    pauli_list = []
    weights = []

    # Create Pauli operators based on ESG scores
    for company in esg_data:
        score = company["esg_score"]
        weights.append(score)
        # Create a corresponding Pauli operator (assuming 4 companies)
        pauli_operator = "I" * len(esg_data)  # Use 'I' for non-target qubits
        pauli_operator = pauli_operator[:len(pauli_operator)-len(weights)] + "Z" * len(weights)
        pauli_list.append([score, pauli_operator])

    return pauli_list

# Setting up Quantum Rings Provider
provider = QuantumRingsProvider(token=os.environ.get('TOKEN_QUANTUMRINGS'), name=os.environ.get('ACCOUNT_QUANTUMRINGS'))
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

print(provider.active_account())

# Define the ansatz (can use SimpleAnsatz or TwoLocalAnsatz)
def SimpleAnsatz(qc, q, n_qubits, theta_list, reps=5, insert_barriers=False):
    for i in range(reps + 1):
        for j in range(n_qubits):
            qc.ry(theta_list[(i * n_qubits) + j], q[j])
        if insert_barriers:
            qc.barrier()
    return

# Define perform_pauli_measurements function
def perform_pauli_measurements(qubitOp, param_dict, SHOTS=1024):
    avg = 0.0
    # Get length of Qbit Operator String
    n_qubits = len(qubitOp[0][1])
    pauli_list = qubitOp

    for p in pauli_list:
        weight = p[0].real
        pauli = p[1]

        # Convert pauli string to integer representation
        # here each character from string is treated as bit and 
        pauli_int = int(p[1].replace("I", "0").replace("Z", "1").replace("X", "1").replace("Y", "1"), 2)

        qc = QuantumCircuit(n_qubits)
        for i in range(n_qubits):
            #Rotation w.r.t Y baed on random value generated [i] -> bit[i]
            qc.ry(param_dict[i], i)

        # Apply Pauli operators
        for i in range(n_qubits):
            if pauli[i] == "Y":
                qc.sdg(i)
                qc.h(i)
            elif pauli[i] == "X":
                qc.h(i)

        qc.measure_all()
        job = backend.run(qc, shots=SHOTS, mode="sync", performance="HighestEfficiency", quiet=True)
        job_monitor(job, quiet=True)

        results = job.result()
        result_dict = results.get_counts()
        print(p, result_dict)
        # Measurement calculation
        measurement = 0.0
        for key, value in result_dict.items():
            #To check
            sign = -1.0 if (bin(int(key, 2) & pauli_int).count("1") & 1) else 1.0
            measurement += sign * value
        measurement /= SHOTS
        measurement *= weight
        avg += measurement

    return avg

# Load ESG data from JSON
esg_data = load_esg_data('./small_company_esgscore.json')
qubitOp = get_Paulioperator(esg_data)

# Initialize parameters
n_qubits = len(esg_data)
param_dict = np.random.rand((n_qubits + 1) * n_qubits)
print('Before:',param_dict)
# Run VQE optimization
for _ in range(100):  # You can adjust the number of iterations as needed
    avg_energy = perform_pauli_measurements(qubitOp, param_dict, SHOTS=shots)
    # print(f"Current energy: {avg_energy}")

# Print the final optimized parameters
print("Optimized parameters:", param_dict)


{'name': 'challenge4_team2@qai-ventures.com', 'token': 'rings-256.BT5Mmd8rd5fPx5K6p0Ff9bVyA9cUDON3', 'max_qubits': '256'}
Before: [0.59946749 0.10871055 0.86695296 0.21499703 0.19491104 0.71071194]
[19.98, 'IZ'] {'00': 931, '01': 90, '10': 3}
[22.06, 'ZZ'] {'00': 945, '01': 78, '10': 1}
[19.98, 'IZ'] {'00': 924, '01': 95, '10': 4, '11': 1}
[22.06, 'ZZ'] {'00': 939, '01': 80, '10': 4, '11': 1}
[19.98, 'IZ'] {'00': 944, '01': 77, '10': 2, '11': 1}
[22.06, 'ZZ'] {'00': 938, '01': 83, '10': 3}
[19.98, 'IZ'] {'00': 924, '01': 96, '10': 4}
[22.06, 'ZZ'] {'00': 931, '01': 88, '10': 4, '11': 1}
[19.98, 'IZ'] {'00': 934, '01': 90}
[22.06, 'ZZ'] {'00': 926, '01': 97, '10': 1}
[19.98, 'IZ'] {'00': 909, '01': 112, '10': 3}
[22.06, 'ZZ'] {'00': 937, '01': 81, '10': 6}
[19.98, 'IZ'] {'00': 929, '01': 91, '10': 4}
[22.06, 'ZZ'] {'00': 921, '01': 98, '10': 5}
[19.98, 'IZ'] {'00': 931, '01': 92, '11': 1}
[22.06, 'ZZ'] {'00': 916, '01': 107, '10': 1}
[19.98, 'IZ'] {'00': 934, '01': 89, '10': 1}
[22.06, 

In [6]:
## this is for esg_score and company value
## Code is missing to optimizet initial parameters input_params=output_params
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
import json
import numpy as np
import os

# Load the JSON data with esg_score and company_value
def load_esg_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Setup the Hamiltonian based on ESG scores and company values
def get_Paulioperator(esg_data):
    pauli_list = []
    weights = []

    # Create Pauli operators based on ESG scores and company values
    for company in esg_data:
        score = company["esg_score"]
        value = company["company_value"]
        
        # Combine esg_score and company_value into a single weight
        weight = score * value
        weights.append(weight)
        
        # Create a corresponding Pauli operator (assuming 4 companies)
        pauli_operator = "I" * len(esg_data)  # Use 'I' for non-target qubits
        pauli_operator = pauli_operator[:len(pauli_operator)-len(weights)] + "Z" * len(weights)
        pauli_list.append([weight, pauli_operator])

    return pauli_list

# Setting up Quantum Rings Provider
provider = QuantumRingsProvider(token=os.environ.get('TOKEN_QUANTUMRINGS'), name=os.environ.get('ACCOUNT_QUANTUMRINGS'))
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

print(provider.active_account())

# Define the ansatz (can use SimpleAnsatz or TwoLocalAnsatz)
def SimpleAnsatz(qc, q, n_qubits, theta_list, reps=5, insert_barriers=False):
    for i in range(reps + 1):
        for j in range(n_qubits):
            qc.ry(theta_list[(i * n_qubits) + j], q[j])
        if insert_barriers:
            qc.barrier()
    return

# Define perform_pauli_measurements function
def perform_pauli_measurements(qubitOp, param_dict, SHOTS=1024):
    avg = 0.0
    n_qubits = len(qubitOp[0][1])
    pauli_list = qubitOp

    for p in pauli_list:
        weight = p[0].real
        pauli = p[1]

        # Convert pauli string to integer representation
        pauli_int = int(p[1].replace("I", "0").replace("Z", "1").replace("X", "1").replace("Y", "1"), 2)

        qc = QuantumCircuit(n_qubits)
        for i in range(n_qubits):
            qc.ry(param_dict[i], i)

        # Apply Pauli operators
        for i in range(n_qubits):
            if pauli[i] == "Y":
                qc.sdg(i)
                qc.h(i)
            elif pauli[i] == "X":
                qc.h(i)

        qc.measure_all()
        job = backend.run(qc, shots=SHOTS, mode="sync", performance="HighestEfficiency", quiet=True)
        job_monitor(job, quiet=True)

        results = job.result()
        result_dict = results.get_counts()

        # Measurement calculation
        measurement = 0.0
        for key, value in result_dict.items():
            sign = -1.0 if (bin(int(key, 2) & pauli_int).count("1") & 1) else 1.0
            measurement += sign * value
        measurement /= SHOTS
        measurement *= weight
        avg += measurement

    return avg

# Load ESG data from JSON with company_value
esg_data = load_esg_data('./small_company_esgscore_company_val.json')
qubitOp = get_Paulioperator(esg_data)

# Initialize parameters
n_qubits = len(esg_data)
param_dict = np.random.rand((n_qubits + 1) * n_qubits)

# Run VQE optimization
for _ in range(100):  # You can adjust the number of iterations as needed
    avg_energy = perform_pauli_measurements(qubitOp, param_dict, SHOTS=shots)
    print(f"Current energy: {avg_energy}")

# Print the final optimized parameters
print("Optimized parameters:", param_dict)


{'name': 'challenge4_team2@qai-ventures.com', 'token': 'rings-256.BT5Mmd8rd5fPx5K6p0Ff9bVyA9cUDON3', 'max_qubits': '256'}
Current energy: 393.1981640625
Current energy: 416.6649609375
Current energy: 402.2171484375
Current energy: 386.23953125
Current energy: 415.9455078125
Current energy: 391.4612109375
Current energy: 399.05582031250003
Current energy: 377.61660156249997
Current energy: 398.23093750000004
Current energy: 396.1560546875
Current energy: 418.71078125
Current energy: 411.79574218749997
Current energy: 428.56917968749997
Current energy: 400.77453125
Current energy: 433.3512109375
Current energy: 397.74398437499997
Current energy: 384.546171875
Current energy: 394.75703124999995
Current energy: 417.1373828125
Current energy: 410.767421875
Current energy: 402.496953125
Current energy: 400.3348828125
Current energy: 392.1080078125
Current energy: 423.05316406250006
Current energy: 421.3307421875
Current energy: 413.325546875
Current energy: 414.1940234375
Current energy: 440

In [12]:
## this is for esg_score and company value
## With code to optimize the vlaues
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
import json
import numpy as np
import os
from scipy.optimize import minimize

# Load the JSON data with esg_score and company_value
def load_esg_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Setup the Hamiltonian based on ESG scores and company values
def get_Paulioperator(esg_data):
    pauli_list = []
    weights = []

    # Create Pauli operators based on ESG scores and company values
    for company in esg_data:
        score = company["esg_score"]
        value = company["company_value"]
        
        # Combine esg_score and company_value into a single weight
        weight = score * value
        weights.append(weight)
        
        # Create a corresponding Pauli operator (assuming 4 companies)
        pauli_operator = "I" * len(esg_data)  # Use 'I' for non-target qubits
        pauli_operator = pauli_operator[:len(pauli_operator)-len(weights)] + "Z" * len(weights)
        pauli_list.append([weight, pauli_operator])

    return pauli_list

# Setting up Quantum Rings Provider
provider = QuantumRingsProvider(token=os.environ.get('TOKEN_QUANTUMRINGS'), name=os.environ.get('ACCOUNT_QUANTUMRINGS'))
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

print(provider.active_account())

# Define the ansatz (can use SimpleAnsatz or TwoLocalAnsatz)
def SimpleAnsatz(qc, q, n_qubits, theta_list, reps=5, insert_barriers=False):
    for i in range(reps + 1):
        for j in range(n_qubits):
            qc.ry(theta_list[(i * n_qubits) + j], q[j])
        if insert_barriers:
            qc.barrier()
    return

# Define perform_pauli_measurements function
def perform_pauli_measurements(qubitOp, param_dict, SHOTS=1024):
    avg = 0.0
    n_qubits = len(qubitOp[0][1])
    pauli_list = qubitOp

    for p in pauli_list:
        weight = p[0].real
        pauli = p[1]

        # Convert pauli string to integer representation
        pauli_int = int(p[1].replace("I", "0").replace("Z", "1").replace("X", "1").replace("Y", "1"), 2)

        qc = QuantumCircuit(n_qubits)
        for i in range(n_qubits):
            qc.ry(param_dict[i], i)

        # Apply Pauli operators
        for i in range(n_qubits):
            if pauli[i] == "Y":
                qc.sdg(i)
                qc.h(i)
            elif pauli[i] == "X":
                qc.h(i)

        qc.measure_all()
        job = backend.run(qc, shots=SHOTS, mode="sync", performance="HighestEfficiency", quiet=True)
        job_monitor(job, quiet=True)

        results = job.result()
        result_dict = results.get_counts()

        # Measurement calculation
        measurement = 0.0
        for key, value in result_dict.items():
            sign = -1.0 if (bin(int(key, 2) & pauli_int).count("1") & 1) else 1.0
            measurement += sign * value
        measurement /= SHOTS
        measurement *= weight
        avg += measurement

    return avg

# Define the cost function for optimization
def cost_function(param_dict, qubitOp, shots):
    avg_energy = perform_pauli_measurements(qubitOp, param_dict, SHOTS=shots)
    return avg_energy

# Load ESG data from JSON with company_value
esg_data = load_esg_data('./small_company_esgscore_company_val.json')
qubitOp = get_Paulioperator(esg_data)

# Initialize parameters (random starting point)
n_qubits = len(esg_data)
param_dict = np.random.rand((n_qubits + 1) * n_qubits)

# Optimization using scipy.optimize.minimize
result = minimize(cost_function, param_dict, args=(qubitOp, shots), method='COBYLA', options={'maxiter': 100})

# Print the final optimized parameters and minimum energy
optimized_param_dict = result.x
min_energy = result.fun

print("Optimized parameters:", optimized_param_dict)
print("Minimum energy:", min_energy)


{'name': 'challenge4_team2@qai-ventures.com', 'token': 'rings-256.BT5Mmd8rd5fPx5K6p0Ff9bVyA9cUDON3', 'max_qubits': '256'}
Optimized parameters: [ 3.10479591  0.00467523 -0.53486852  0.51636263  0.09857276  2.84850829]
Minimum energy: -768.38


In [18]:
# This code will return best compnay that one can invest in 
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
import json
import numpy as np
import os
from scipy.optimize import minimize

# Load the JSON data with esg_score and company_value
def load_esg_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Setup the Hamiltonian based on ESG scores and company values
def get_Paulioperator(esg_data):
    pauli_list = []
    weights = []

    # Create Pauli operators based on ESG scores and company values
    for company in esg_data:
        score = company["esg_score"]
        value = company["company_value"]
        
        # Combine esg_score and company_value into a single weight
        weight = score * value
        weights.append(weight)
        
        # Create a corresponding Pauli operator (assuming 4 companies)
        pauli_operator = "I" * len(esg_data)  # Use 'I' for non-target qubits
        pauli_operator = pauli_operator[:len(pauli_operator)-len(weights)] + "Z" * len(weights)
        pauli_list.append([weight, pauli_operator])

    return pauli_list

# Setting up Quantum Rings Provider
provider = QuantumRingsProvider(token=os.environ.get('TOKEN_QUANTUMRINGS'), name=os.environ.get('ACCOUNT_QUANTUMRINGS'))
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

# print(provider.active_account())

# Define the ansatz (can use SimpleAnsatz or TwoLocalAnsatz)
def SimpleAnsatz(qc, q, n_qubits, theta_list, reps=5, insert_barriers=False):
    for i in range(reps + 1):
        for j in range(n_qubits):
            qc.ry(theta_list[(i * n_qubits) + j], q[j])
        if insert_barriers:
            qc.barrier()
    return

# Define perform_pauli_measurements function
def perform_pauli_measurements(qubitOp, param_dict, SHOTS=1024):
    avg = 0.0
    n_qubits = len(qubitOp[0][1])
    pauli_list = qubitOp

    for p in pauli_list:
        weight = p[0].real
        pauli = p[1]

        # Convert pauli string to integer representation
        pauli_int = int(p[1].replace("I", "0").replace("Z", "1").replace("X", "1").replace("Y", "1"), 2)

        qc = QuantumCircuit(n_qubits)
        for i in range(n_qubits):
            qc.ry(param_dict[i], i)

        # Apply Pauli operators
        for i in range(n_qubits):
            if pauli[i] == "Y":
                qc.sdg(i)
                qc.h(i)
            elif pauli[i] == "X":
                qc.h(i)

        qc.measure_all()
        job = backend.run(qc, shots=SHOTS, mode="sync", performance="HighestEfficiency", quiet=True)
        job_monitor(job, quiet=True)

        results = job.result()
        result_dict = results.get_counts()

        # Measurement calculation
        measurement = 0.0
        for key, value in result_dict.items():
            sign = -1.0 if (bin(int(key, 2) & pauli_int).count("1") & 1) else 1.0
            measurement += sign * value
        measurement /= SHOTS
        measurement *= weight
        avg += measurement

    return avg

# Define the cost function for optimization
def cost_function(param_dict, qubitOp, shots):
    avg_energy = perform_pauli_measurements(qubitOp, param_dict, SHOTS=shots)
    return avg_energy

# Function to calculate individual company energies
def calculate_company_energies(optimized_param_dict, esg_data):
    company_energies = []
    
    for i, company in enumerate(esg_data):
        company_weight = company['esg_score'] * company['company_value']
        pauli_op = [[company_weight, "Z" + "I" * (len(esg_data) - 1)]]
        
        energy = perform_pauli_measurements(pauli_op, optimized_param_dict, SHOTS=1024)
        company_energies.append({
            "company": company["company"],
            "energy": energy
        })
    
    return company_energies

# Load ESG data from JSON with company_value
esg_data = load_esg_data('./small_company_esgscore_company_val.json')
qubitOp = get_Paulioperator(esg_data)

# Initialize parameters (random starting point)
n_qubits = len(esg_data)
param_dict = np.random.rand((n_qubits + 1) * n_qubits)
# Optimization using scipy.optimize.minimize
result = minimize(cost_function, param_dict, args=(qubitOp, shots), method='COBYLA', options={'maxiter': 100})

# Get optimized parameters and minimum energy
optimized_param_dict = result.x
min_energy = result.fun

# print("Optimized parameters:", optimized_param_dict)
# print("Minimum energy:", min_energy)

# Calculate company-specific energies
company_energies = calculate_company_energies(optimized_param_dict, esg_data)

# Find the company closest to the minimum energy
# closest_company = min(company_energies, key=lambda x: abs(x["energy"] - min_energy))
# print(f"Company closest to minimum energy: {closest_company['company']} with energy: {closest_company['energy']}")

return sorted(company_energies,key=lambda x: abs(x["energy"] - min_energy))

{'name': 'challenge4_team2@qai-ventures.com', 'token': 'rings-256.BT5Mmd8rd5fPx5K6p0Ff9bVyA9cUDON3', 'max_qubits': '256'}
Optimized parameters: [ 3.1223851  -0.02721241  0.13616411 -0.13547924 -0.2661444  -0.26144548]
Minimum energy: -768.38
[{'company': 'PepsiCo, Inc.', 'energy': 308.84}, {'company': 'Broadcom Inc.', 'energy': 459.54}]


In [27]:
import sample_function
sample_function.fun()

This is fun
